# 《Problem Solving with Algorithms and Data Structures using Python》 的学习笔记和课后作业答案（六. Trees and Tree Algorithms）

```
对应本书第六章。
```

## 目录

* [6.Trees and Tree Algorithms](#6.Trees-and-Tree-Algorithms)
    + [笔记](#笔记)
        + [用结点和引用实现的二叉树](#用结点和引用实现的二叉树)
        + [解析树](#解析树)
        + [树的遍历](#树的遍历)
        + [选择排序](#选择排序)
        + [插入排序](#插入排序)
        + [希尔排序](#希尔排序)
        + [合并排序](#合并排序)
        + [快速排序](#快速排序)
    + [作业](#作业)
        + [q1](#q1)
        + [q2](#q2)
        + [q4](#q4)
        + [q9](#q9)

# 6.Trees and Tree Algorithms

[原目录](http://interactivepython.org/courselib/static/pythonds/Trees/toctree.html)

## 笔记

作者介绍了两个计算机中的树结构：文件系统（File System）和HTML页面。

### 用结点和引用实现的二叉树


In [1]:
from __future__ import print_function

In [2]:
class BinaryTree:
    def __init__(self,rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self,newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    def insertRight(self,newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t


    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild

    def setRootVal(self,obj):
        self.key = obj

    def getRootVal(self):
        return self.key

In [3]:
r = BinaryTree('a')
print(r.getRootVal())
print(r.getLeftChild())
r.insertLeft('b')
print(r.getLeftChild())
print(r.getLeftChild().getRootVal())
r.insertRight('c')
print(r.getRightChild())
print(r.getRightChild().getRootVal())
r.getRightChild().setRootVal('hello')
print(r.getRightChild().getRootVal())

a
None
<__main__.BinaryTree instance at 0x7f55e658a320>
b
<__main__.BinaryTree instance at 0x7f55e33bedd0>
c
hello


### 解析树

Parse　Tree的构建规则（这里是数学表达式的解析树）：

- 如果当前token是`'('`，增加一个结点作为左子结点，下降到左子结点。
- 如果当前token属于`['+','-','/','*']`，将当前结点的值设为这个操作符，增加一个结点作为右子结点，下降到右子结点。
- 如果当前token是一个数字，则将当前结点的值设为这个数字，回到父结点。
- 如果当前token是`')'`，回到父结点。

构建的解析树如下：

![](http://interactivepython.org/courselib/static/pythonds/_images/buildExp8.png)

构建解析树的代码：

In [6]:
from pythonds.basic.stack import Stack
from pythonds.trees.binaryTree import BinaryTree

def buildParseTree(fpexp):
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree
    for i in fplist:
        # 如果当前token是`'('`，增加一个结点作为左子结点，下降到左子结点。
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        # 如果当前token是一个数字，则将当前结点的值设为这个数字，回到父结点。
        elif i not in ['+', '-', '*', '/', ')']:
            currentTree.setRootVal(int(i))
            parent = pStack.pop()
            currentTree = parent
        # 如果当前token属于`['+','-','/','*']`，将当前结点的值设为这个操作符，增加一个结点作为右子结点，下降到右子结点。
        elif i in ['+', '-', '*', '/']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()
        # 如果当前token是`')'`，回到父结点。
        elif i == ')':
            currentTree = pStack.pop()
        else:
            raise ValueError
    return eTree

pt = buildParseTree("( 3 + ( 4 * 5 ) )")
pt.postorder()  #defined and explained in the next section

3
4
5
*
+


计算表达式的值：

In [7]:
import operator
def evaluate(parseTree):
    opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv}

    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()

    if leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC),evaluate(rightC))
    else:
        return parseTree.getRootVal()
evaluate(pt)

23

### 树的遍历

- 1.前序遍历（preorder），即根左右。
```python
def preorder(tree):
    if tree:
        print(tree.getRootVal())
        preorder(tree.getLeftChild())
        preorder(tree.getRightChild())
```

- 2.中序遍历（inorder），即左根右。
```python
def inorder(tree):
  if tree != None:
      inorder(tree.getLeftChild())
      print(tree.getRootVal())
      inorder(tree.getRightChild())
```

- 3.后序遍历（postorder），即左右根。
```python
def postorder(tree):
    if tree != None:
        postorder(tree.getLeftChild())
        postorder(tree.getRightChild())
        print(tree.getRootVal())
```

- 4.层次遍历，即：根/子子/孙孙孙孙

原书没有提及层次遍历，这里做一点补充。

In [9]:
def level_order(tree):
    # 维护一个队列
    trav_queue = []
    trav_queue.insert(0, tree)
    while len(trav_queue) != 0:
        cur_node = trav_queue.pop()
        if cur_node.getLeftChild():
            trav_queue.insert(0, cur_node.getLeftChild())
        if cur_node.getRightChild():
            trav_queue.insert(0, cur_node.getRightChild())
        print(cur_node.getRootVal())

In [8]:
# 对比前序遍历和层次遍历的区别
def preorder(tree):
    if tree:
        print(tree.getRootVal())
        preorder(tree.getLeftChild())
        preorder(tree.getRightChild())
r = BinaryTree('a')
r.insertLeft('b')
r.insertRight('c')
r.getLeftChild().insertLeft('d')
r.getLeftChild().insertLeft('e')
preorder(r)

a
b
e
d
c


In [10]:
level_order(r)

a
b
c
e
d
